In [1]:
import os
from PIL import Image

In [2]:
dirs = ['parks', 'beard', 'Atanasof', 'Catt', 'gillman', 'mu', 'Mckay']
str_to = {i:dirs[i] for i in range(len(dirs))}

In [41]:
pics= []
targets = []

In [42]:
ind = 0

for d in dirs:
    cnt = 0
    for f in os.listdir(os.getcwd() + '/' + d):
        file_name = os.getcwd() + '/' + d + '/' + f
        try:
            pics.append(Image.open(file_name))
            targets.append(ind)
            cnt += 1
        except Exception:
            pass
        
    print(d + ' ' +  str(cnt))
    ind += 1

parks 104
beard 42
Atanasof 11
Catt 40
gillman 81
mu 45
Mckay 57


In [10]:
import tensorflow as tf
import numpy as np

In [46]:
new_dims = (50, 30)

In [51]:
pics[1].resize(new_dims)

In [52]:
new_pics = [x.resize(new_dims) for x in pics]

In [53]:
#import pickle
#pickle.dump(new_pics, open('50by30', mode='wb+'))
# pickle.dump(targets, open('targets', mode='wb+'))

In [3]:
import pickle
new_dims = (50, 30)
new_pics = pickle.load(open('50by30', mode='rb'))
targets = pickle.load(open('targets', mode='rb'))

In [4]:
new_pics[5]

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
X_train, X_test, y_train, y_test = train_test_split(new_pics, targets, stratify=targets)

In [103]:
len(X_train)

285

In [104]:
batch_size = 15 # 285 / 5
num_target = len(dirs)

In [193]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, shape=(None, *new_dims, 3))


y = tf.placeholder_with_default(np.zeros((batch_size), dtype='int32'),  shape=(batch_size))

is_training = tf.placeholder(tf.bool)

layer = tf.layers.conv2d(X, filters=45, kernel_size=(2,2), strides=(1,1))
layer = tf.nn.relu(layer)



layer = tf.layers.conv2d(layer, filters=45, kernel_size=(2,2), strides=(1,1))
layer = tf.nn.relu(layer)

layer = tf.layers.max_pooling2d(layer, (2, 2), (2, 2))

layer = tf.layers.conv2d(layer, filters=45, kernel_size=(3,3), strides=(1,1))
layer = tf.nn.relu(layer)

print(layer)

layer = tf.layers.dense(tf.reshape(layer, (-1, 12 * 22 * 45)), num_target)

predictions = tf.argmax(layer, axis=1)


loss = tf.losses.softmax_cross_entropy(tf.one_hot(y, num_target), layer)


extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

opt = tf.train.AdamOptimizer().minimize(loss)

Tensor("Relu_2:0", shape=(?, 22, 12, 45), dtype=float32)


In [216]:
X

<tf.Tensor 'Placeholder:0' shape=(?, 50, 30, 3) dtype=float32>

In [195]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [196]:
X_train_real = [list(x.getdata()) for x in X_train]
X_test_real = [list(x.getdata()) for x in X_test]

In [197]:
batch_size

15

In [198]:
total_iters = 0
cur = 0

In [199]:
len(X_test_real)

95

In [200]:
for i in range(301):
    
    total_iters += 1
    
    if cur + batch_size >= len(X_train):
        cur = 0
            
    X_cur = np.asarray(X_train_real[cur:cur+batch_size]).reshape(batch_size, *new_dims,3)
    y_cur = y_train[cur:cur+batch_size]
    
    feed = {X : X_cur, y:y_cur, is_training:True}
    loss_, opt_ = sess.run([loss, opt], feed)
    
    cur += batch_size
    
    if i % 10 == 0:
        pos = 0
        ll = 0

        for i in range(6):

            X_cur = np.asarray(X_test_real[pos:pos+batch_size]).reshape(batch_size, *new_dims,3)
            y_cur = y_test[pos:pos+batch_size]
            pos += batch_size
            feed = {X : X_cur, y:y_cur, is_training:False}
            loss_, predictions_ = sess.run([loss, predictions], feed)
            ll += loss_

        print('valiadation loss %f' % ll)
        print()

    print('iteration %d : loss % f' % (total_iters, loss_))

valiadation loss 393.710907

iteration 1 : loss  43.885380
iteration 2 : loss  56.839378
iteration 3 : loss  56.119072
iteration 4 : loss  128.266891
iteration 5 : loss  176.811005
iteration 6 : loss  82.758499
iteration 7 : loss  121.331917
iteration 8 : loss  38.022079
iteration 9 : loss  20.137838
iteration 10 : loss  10.546154
valiadation loss 43.964803

iteration 11 : loss  4.919749
iteration 12 : loss  6.947706
iteration 13 : loss  4.452776
iteration 14 : loss  4.953309
iteration 15 : loss  2.436296
iteration 16 : loss  2.676634
iteration 17 : loss  2.050520
iteration 18 : loss  1.814793
iteration 19 : loss  1.567325
iteration 20 : loss  1.514025
valiadation loss 9.755829

iteration 21 : loss  1.541753
iteration 22 : loss  1.373916
iteration 23 : loss  1.476927
iteration 24 : loss  1.340032
iteration 25 : loss  1.293217
iteration 26 : loss  1.336618
iteration 27 : loss  1.419286
iteration 28 : loss  1.238998
iteration 29 : loss  1.126410
iteration 30 : loss  1.748558
valiadation 

KeyboardInterrupt: 

In [209]:
pos = 15

X_cur = np.asarray(X_test_real[pos:pos+batch_size]).reshape(batch_size, *new_dims,3)
y_cur = y_test[pos:pos+batch_size]

feed = {X : X_cur, y:y_cur, is_training:False}
loss_, predictions_ = sess.run([loss, predictions], feed)

In [210]:
y_cur

[5, 0, 0, 4, 1, 6, 5, 0, 0, 0, 3, 6, 1, 0, 4]

In [211]:
wrong = y_cur == predictions_
print('%d / %d' % (np.sum(wrong), batch_size))
print(wrong)

12 / 15
[ True  True False  True  True  True  True  True  True  True  True  True
 False False  True]


In [212]:
j = 3
print(wrong[j], dirs[predictions_[j]])
X_test[pos+j]

True gillman


In [213]:
loss_, predictions_

(0.82356006, array([5, 0, 6, 4, 1, 6, 5, 0, 0, 0, 3, 6, 0, 5, 4]))

In [215]:
builder = tf.saved_model.builder.SavedModelBuilder(os.path.join(
      tf.compat.as_bytes('/tmp'),
      tf.compat.as_bytes(str(1)))
)

tensor_info_x = tf.saved_model.utils.build_tensor_info(X)
tensor_info_y = tf.saved_model.utils.build_tensor_info(layer)

prediction_signature = (tf.saved_model.signature_def_utils.build_signature_def(
          inputs={'images': tensor_info_x},
          outputs={'scores': tensor_info_y},
          method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME))

legacy_init_op = tf.group(tf.tables_initializer(), name='legacy_init_op')

builder.add_meta_graph_and_variables(sess, [tf.saved_model.tag_constants.SERVING],
      signature_def_map={
          'predict_images':
              prediction_signature
      },
      legacy_init_op=legacy_init_op)

builder.save()

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: b'/tmp/1/saved_model.pb'


b'/tmp/1/saved_model.pb'